In [3]:
import pandas as pd
import requests
#from ../
#from api_utils import loadCSVData,updateIncidenceTable
#from api_utils import getRawDataToCSV
#from api_utils import buildbrutIncidence,incidenceMovingAverage,incidenceExpMovingAverage
pd.options.mode.chained_assignment = None

In [4]:
continent_map = {
    'Europe' : { 
        "France":"France",
        "Germany":"Allemagne",   
        "Belgium":"Belgique",
        "Spain":"Espagne",
        "Greece":"Grèce",
        "Ireland":"Ireland",
        "Italy":"Italie",
        "Netherlands":"Pays-bas",
        "Poland":"Pologne",
        "Ukraine":"Ukraine",
        "United Kingdom":"Royaume-unis",
        "Sweden":"Suède",
        "Switzerland":"Suisse",
        "Turkey":"Turquie",
    },
    'America' : { 
        "United States of America":"Etats-unis",
        "Mexico" : "Mexique",
        "Canada" : "Canada", #bugg request return empty
        "Brazil":"Brazil",
        "Colombia":"Colombia",
        "Argentina":"Argentina",
        "Peru":"Perou",

    },
    'Asia' : {
        "Afghanistan":"Afghanistan",
        "India":"Inde",
        "Indonesia":"Indonésie",
        "Bangladesh":"Bangladesh",
        "China":"Chine",
        "Japan":"Japon",
        "Myanmar":"Byrmanie",
        "Pakistan":"Pakistan",
        "Philippines":"Philippines",
        "Russian Federation":"Russie",
        "Thailand":"Thailande",
        "Uzbekistan":"Uzbekistan",
        "Malaysia":"Malaisie",
        "Nepal":"Nepal",


    },
    'Africa' : {
        "Algeria":"Algeria",
        "Iraq":"Iraq",
        "South Africa" : "Afrique du sud",
        "Morocco":"Maroc",
        "Nigeria":"Nigeria",
        "Ethiopia":"Ethiopia",
        "Kenya":"Kenya",
        "Sudan":"Sudan",
        "Saudi Arabia":"Arabie Saoudite",
        "Uganda":"Uganda",
        "Angola":"Angola",
        "Mozambique":"Mozambique",
        "Ghana":"Ghana",
        "Madagascar":"Madagascar",
        "Cameroon":"Cameroon"

    },
}

continentslist = ['Europe','America','Asia','Africa']

def get_countrymap():
    resultdict = {}
    for name in continentslist:
        somedict = {k:v for (k,v) in continent_map[name].items()}
        resultdict.update(somedict)
    return resultdict

country_map = get_countrymap()

In [7]:
import time
def getPandaFrameForCountry(CountryName):
    """
        return full data off covid19api for input CountryName
        JSON format to transform in dataframe
    """
    # API config
    url = "https://api.covid19api.com/total/country/"+CountryName
    payload = {}
    headers= {} 

    # request loop config
    Rcode = 0
    trycount = 0
    while Rcode != 200 and trycount < 5:
        response = requests.request("GET", url, headers=headers, data = payload)
        Rcode = response.status_code
        if Rcode != 200 :
            try : message = response.json()
            except : message = ''
            print(Rcode,message)
            time.sleep(4)  
        else : print('requests {} for {}'.format(Rcode,CountryName))
        trycount += 1

    return response.json()

In [8]:
for i in country_map.keys():
    rawdf=pd.DataFrame(getPandaFrameForCountry(i))
    if i == 'Canada' : print(rawdf.head(1))

requests 200 for France
requests 200 for Germany
requests 200 for Belgium
requests 200 for Spain
requests 200 for Greece
429 {'message': 'Too Many Requests on /total/country/Ireland. Please upgrade to a subscription at https://covid19api.com/#subscribe', 'success': False}
requests 200 for Ireland
requests 200 for Italy
requests 200 for Netherlands
requests 200 for Poland
requests 200 for Ukraine
requests 200 for United Kingdom
requests 200 for Sweden
requests 200 for Switzerland
requests 200 for Turkey
requests 200 for United States of America
429 {'message': 'Too Many Requests on /total/country/Mexico. Please upgrade to a subscription at https://covid19api.com/#subscribe', 'success': False}
requests 200 for Mexico
requests 200 for Canada
  Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Canada                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
requests 2

KeyboardInterrupt: 

In [71]:
paysname = 'Canada'
test = getRawDataToCSV(paysname)
test.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [74]:
import requests
def getPandaFrameForCountry(CountryName):
    """
        return full data off covid19api for input CountryName
        JSON format to transform in dataframe
    """
    url = "https://api.covid19api.com/total/country/"+CountryName
    payload = {}
    headers= {}
    response = requests.request("GET", url, headers=headers, data = payload)
    return response.json()
test = getPandaFrameForCountry('Canada')
test

[]

In [69]:
updateIncidenceTable(paysname)

ERROR : file not found workdata/raw_Canada_ALLTable.csv
ERROR : file not found workdata/raw_Canada_ALLTable.csv


FileNotFoundError: 

In [42]:
countrylist = ['France','United Kingdom','China']
seriesdict = {}
for elem in countrylist : 
    updateIncidenceTable(elem)
    seriesdict[elem] = loadCSVData(elem,'Inc')['Confirmed_eMMincidence']

no recent data detected for France, incidence update cancelled
no recent data detected for United Kingdom, incidence update cancelled
ERROR : file not found workdata/raw_China_ALLTable.csv
ERROR : file not found workdata/incidence_China_Table.csv
new data detected : 363 more lines


In [43]:
print(seriesdict['France'])

0          0.00
1          0.00
2          0.20
3          0.28
4          0.25
         ...   
358    16882.49
359    17361.54
360    15654.38
361    17829.45
362    16046.50
Name: Confirmed_eMMincidence, Length: 363, dtype: float64


In [44]:
dfpop = pd.read_csv("workdata/population_2019.csv",index_col=0)
dfpop.head()

,population
country,
Afghanistan,38
Albania,2.9
Algeria,43.1
American Samoa,0.1
Andorra,0.1


In [45]:
listoflist = []
for name in countrylist :  
        # starting with inc per million > 10
        testserie = seriesdict[name]/(float(dfpop['population'][name]))
        testserie = testserie.loc[(testserie>10)].reset_index(drop=True)
        #testserie = testserie    


        testdf = pd.DataFrame(testserie.tolist(),columns=[name])
        listoflist.append(testdf)
    
comparativeinc= pd.concat(listoflist, axis=1)
#comparativeinc = pd.DataFrame(listoflist,columns=countrylist)
comparativeinc.head()

,France,United Kingdom,China
0,11.184501,10.117665,NaN
1,12.730700,12.622305,NaN
2,13.998659,14.926048,NaN
3,15.251565,17.500749,NaN
4,19.508793,20.431886,NaN


In [32]:
testserie = seriesdict['United Kingdom'].loc[(seriesdict['United Kingdom']>100)].reset_index(drop=True)
print(testserie)
#testserie = testserie.apply(lambda x : (float(dfpop['population']['France'])*1000000)/x)
testserie = testserie/(float(dfpop['population']['United Kingdom']))
print(testserie)

0        100.09
1        138.98
2        173.08
3        192.17
4        217.25
         ...   
309    49820.98
310    50427.38
311    49527.44
312    48441.70
313    47358.53
Name: Confirmed_eMMincidence, Length: 314, dtype: float64
0        1.498353
1        2.080539
2        2.591018
3        2.876796
4        3.252246
          ...    
309    745.823054
310    754.900898
311    741.428743
312    725.175150
313    708.960030
Name: Confirmed_eMMincidence, Length: 314, dtype: float64


In [ ]:
# construct table per month for currentyear
listoflist = []
for month in moisdict.values():
    seriemonth = maDataFrame['totalnof'].loc[(maDataFrame['mois']==month)].tolist()
    monthframe = pd.DataFrame(seriemonth,columns=[month])
    listoflist.append(monthframe)
monthframe= pd.concat(listoflist, axis=1)
monthframe.to_csv('publidash/dfSB_distri_camois.csv',index=False) 

In [61]:
dfpop = pd.read_csv("workdata/population_2019.csv",index_col=0)
dfpop.head()

,population
country,
Afghanistan,38
Albania,2.9
Algeria,43.1
American Samoa,0.1
Andorra,0.1


In [62]:
dfapi = pd.read_csv("workdata/countriesNameTable.csv",index_col=1)
print(dfapi.columns)
dfapi.drop(columns=[dfapi.columns[0]],inplace=True)
dfapi.head()

Index(['Unnamed: 0', 'slug', 'ISO2'], dtype='object')


,slug,ISO2
country,,
Peru,peru,PE
Andorra,andorra,AD
Bahamas,bahamas,BS
Gabon,gabon,GA
Kazakhstan,kazakhstan,KZ


In [66]:
testcountry= pd.concat([dfpop,dfapi], axis=1,join='inner')
testcountry.head(20)

,population,slug,ISO2
country,,,
Afghanistan,38,afghanistan,AF
Albania,2.9,albania,AL
Algeria,43.1,algeria,DZ
American Samoa,0.1,american-samoa,AS
Andorra,0.1,andorra,AD
Angola,31.8,angola,AO
Antigua and Barbuda,0.1,antigua-and-barbuda,AG
Argentina,44.9,argentina,AR
Armenia,3,armenia,AM


In [67]:
testcountry.to_csv('workdata/country_concat_inner.csv') 